## Towards Human-Centric Transportation and Energy Metrics: Influence of Mode, Vehicle Occupancy, Trip Distance, and Fuel Economy

This notebook is a companion to the related paper of the same name. We publish our code for greater reproducibility and as a basis for additional innovation in this field. This notebook contains interactive graphs that can be manipulated to explore the metric space. This allows the notebook to be used by practioners as well without modifying any code.

Abstract: 

_Traditional metrics measuring transportation and energy outcomes can be augmented to better represent impacts on people's lives and systems-level performance. In this context, this study introduces two novel metrics: road capacity (as number of people traveling and accessing services) and energy intensity (as energy use for people traveling and accessing services). Current national-level distributions of available data in the United States for factors contributing to the two new integrated metrics are used as context to evaluate potential outcomes. These factors include vehicle occupancy, mode share, fuel economy, and trip distance. Variations in input values provide insights on how these factors shape efficiencies in road capacity and energy intensity. Parametric sensitivity analysis indicates that the impact of each input depends upon the metric being evaluated. For the human-centered road capacity mobility metric, increasing vehicle occupancy has the largest effect &ndash; twice that of increasing mode share for bike, walk, and transit. For the energy intensity mobility metric, the effect of improving fuel economy is the largest. However, when the focus is on accessibility (instead of mobility), for both metrics the effect of lowering average trip distance is the largest. Additionally, a novel interactive tool to visualize the results for various parameter combinations makes the metrics practitioner ready. The findings suggest that the diffusion of new human-centric metrics that benchmark outcomes associated with road capacity and energy may be significant in motivating new sustainable transportation investments and efficient utilization of infrastructure, mobility assets, and services._

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from copy import deepcopy

import numpy as np
import matplotlib.patches as mpatches

from functions.vphpl_pphpl_calc import vphpl_pphpl_calc, pphpl_by_mode
from functions.plots_functions import violin, pdf_plot, cdf_plot, cdf_subplots, calc_ms_pvt
from functions.graphing_functions import cdf_plot, calculate_peit_var, get_data, create_sliders, change_mean_from_kwargs

from ipywidgets import interactive, Layout
%matplotlib inline
import matplotlib as mpl
import seaborn as sns

mpl.rcParams['figure.figsize'] = [8.0, 6.0]
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100

mpl.rcParams['font.size'] = 13
mpl.rcParams['axes.labelsize'] = 13
mpl.rcParams['axes.titlesize'] = 13
mpl.rcParams['legend.fontsize'] = 'medium'
mpl.rcParams['figure.titlesize'] = 'medium'
#mpl.rcParams['figure.subplot.wspace'] = 0.25 #Use when producing 3-wide subplots


#Color-blind friendly for line plots: https://gist.github.com/thriveth/8560036
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']


In [ ]:
params, results, summary = get_data()

## PEIT (Personal Energy Intensity for Transportation (PEIT)

In [ ]:
# ms: mode share
# occ: average occupancy
# FE: fuel efficiency
def peit1_calc(ms, occ, FE):
    return ms * (1/occ) * (1/FE)

In [ ]:
baseline = results['peit1']['combined']

In [ ]:
# summary

In [ ]:
def get_modified_peit(**kwargs):
    # let's say that the original mean was 10, new mean is 20. we want to add 10 to all values
    change_mean = lambda param_mode: change_mean_from_kwargs(param_mode, kwargs)
    
    ms_wb_mod = change_mean("ms_wb")
    ms_transit_mod = change_mean("ms_bus")
    ms_pvt_mod = 1 - (ms_transit_mod + ms_wb_mod)

    pvt_peit_mod = peit1_calc(ms_pvt_mod, change_mean("occ_pvt"), change_mean("FE_pvt"))
    bus_peit_mod= peit1_calc(ms_transit_mod, change_mean("occ_bus"), change_mean("FE_bus"))
    wb_peit_mod=peit1_calc(ms_wb_mod,1,110)
    modified_peit = pvt_peit_mod + bus_peit_mod + wb_peit_mod
    return modified_peit

In [ ]:
def explore_peit_var(**kwargs):
    modified_peit = get_modified_peit(**kwargs)
    cdf_plot([results['peit1']['combined'],modified_peit],['Baseline','Modified'],"PEIT1 [L/km]",['gold','red'])

In [ ]:
PEIT_SLIDER_LIST = ["ms_wb", "ms_bus", "occ_bus", "occ_pvt", "FE_bus", "FE_pvt"]

In [ ]:
# create_sliders()

In [ ]:
# interact_manual(explore_peit_var, **create_sliders(PEIT_SLIDER_LIST));

In [ ]:
interactive_plot_1 = interactive(explore_peit_var, {'manual' : True, 'manual_name' : 'Recalculate'}, **create_sliders(PEIT_SLIDER_LIST));
interactive_plot_2 = interactive(explore_peit_var, {'manual' : True, 'manual_name' : 'Recalculate'}, **create_sliders(PEIT_SLIDER_LIST));
two_graphs = widgets.HBox(children=[interactive_plot_1, interactive_plot_2])
two_graphs

# Everything below this needs to be modified by Dustin

# Standard Deviation Plots for PPHPL

In [ ]:
#PPHPL1 and 2 plots (combined standard deviation CDF curves)
cdf_plot([results['pphpl1']['combined'],calculate_pphpl_var(0,1),calculate_pphpl_var(1,0)],
         ['Baseline','Higher mode share\n(transit bus, walk & bike)','Higher occupancy\n(Private vehicle, Transit bus)'],
         "People per hour per lane",CB_color_cycle)
cdf_plot([results['pphpl2']['combined'],calculate_pphpl2_var(0,1,0),calculate_pphpl2_var(1,0,0),calculate_pphpl2_var(0,0,1)],
         ['Baseline','Higher mode share\n(transit bus, walk & bike)','Higher occupancy\n(Private vehicle, Transit bus)','Lower trip distance'],
         "People per hour per lane (20 km)",CB_color_cycle)

### COMBINED VPHPL and PPHPL plot

In [ ]:
#COMBINED VPHPL AND PPHPL PLOT
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
labels = []
def add_label(violin, label):
    color = violin["bodies"][0].get_facecolor().flatten()
    labels.append((mpatches.Patch(color=color), label))

add_label(plt.violinplot([results["vphpl"]["pvt"],results["vphpl"]["bus"]]), "VPHPL")    
add_label(plt.violinplot([results["pphpl1"]["pvt"],results["pphpl1"]["bus"],results["pphpl1"]["wb"]]), "PPHPL1")
add_label(plt.violinplot([results["pphpl2"]["pvt"],results["pphpl2"]["bus"],results["pphpl2"]["wb"]]), "PPHPL2 (20 km)")
ax.set_yscale("log")
xticklabels = ['Private vehicle', 'Transit bus', 'Walk & bike']
ax.set_xticks([1,2,3])
ax.set_xticklabels(xticklabels)
plt.legend(*zip(*labels), loc=2, fontsize = 12)
ax.set_ylabel('Number of vehicles or people per hour per lane')

In [ ]:
#COMBINED VPHPL AND PPHPL PLOT
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
labels = []
def add_label(violin, label):
    color = violin["bodies"][0].get_facecolor().flatten()
    labels.append((mpatches.Patch(color=color), label))

add_label(plt.violinplot([results["vphpl"]["pvt"],results["vphpl"]["bus"]]), "VPHPL")    
add_label(plt.violinplot([results["pphpl1"]["pvt"],results["pphpl1"]["bus"],results["pphpl1"]["wb"]]), "PPHPL")
ax.set_yscale("log")
xticklabels = ['Private vehicle', 'Transit bus', 'Walk & bike']
ax.set_xticks([1,2,3])
ax.set_xticklabels(xticklabels)
plt.legend(*zip(*labels), loc=2, fontsize = 12)
ax.set_ylabel('Number of vehicles or people per hour per lane')

# Standard Deviation Plots for PEIT (interactive)

In [ ]:
def explore_peit_var(ms_var, occ_var, FE_var):
    cdf_plot([results['peit1']['combined'],calculate_peit_var(ms_var, occ_var, FE_var)],['Baseline','Modified'],"PEIT1 [L/km]",['gold','red'])
    
interactive_plot = interactive(explore_peit_var, ms_var=(-3.0, 3.0,0.5), occ_var=(-3, 3, 0.5), FE_var=(-3,3,0.5))
output = interactive_plot.children[-1]
output.layout.height = '640px'
interactive_plot

### Interaction

In [ ]:
@interact(ms=widgets.FloatSlider(min=0,max=1.0, value=1.0), avo=widgets.IntSlider(min=1, max=5, step=1, value=1), ei=widgets.BoundedIntText(min=1, value=10))
def mpec_driving(ms, avo, ei):
    return 1.0 / (ms * (1/avo) * ei)

In [ ]:
def print_mpec_driving(avo):
    print(mpec_driving(ms=1.0, avo=avo, ei=10))
play = widgets.Play(
    value=1,
    min=1,
    max=5,
    step=1,
    description="Press play",
    disabled=False
)
slider = widgets.FloatSlider(min=1, max=5, step=0.5, description="avo")
widgets.jslink((play, 'value'), (slider, 'value'))
controls = widgets.HBox([play, slider])
out = widgets.interactive_output(print_mpec_driving, {'avo': play})
display(controls, out)

## Additional advanced functionality examples

### Arguments that are dependent on each other

Arguments that are dependent on each other can be expressed manually using `observe`.  See the following example, where one variable is used to describe the bounds of another.  For more information, please see the [widget events example notebook](./Widget%20Events.ipynb).

In [ ]:
x_widget = widgets.FloatSlider(min=0.0, max=10.0, step=0.05)
y_widget = widgets.FloatSlider(min=0.5, max=10.0, step=0.05, value=5.0)

def update_x_range(*args):
    x_widget.max = 2.0 * y_widget.value
y_widget.observe(update_x_range, 'value')

def printer(x, y):
    print(x, y)
interact(printer,x=x_widget, y=y_widget);

### Flickering and jumping output

On occasion, you may notice interact output flickering and jumping, causing the notebook scroll position to change as the output is updated. The interactive control has a layout, so we can set its height to an appropriate value (currently chosen manually) so that it will not change size as it is updated.


In [ ]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(m, b):
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, m * x + b)
    plt.ylim(-5, 5)
    plt.show()

interactive_plot = interactive(f, m=(-2.0, 2.0), b=(-3, 3, 0.5))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot